In [12]:
import requests
import pandas as pd
import time

In [13]:
# access spending_by_award file and create data frame
datafile = 'awards_df.csv'
awards_df = pd.read_csv(datafile)
awards_df['Recipient DUNS Number'] = awards_df['Recipient DUNS Number'].astype(str).str.zfill(9)
awards_df = awards_df.set_index('Recipient DUNS Number')
del awards_df['Unnamed: 0']
awards_df.head()

,Award ID,generated_internal_id,Recipient Name,Base Obligation Date,Start Date,End Date,Place of Performance State Code,Award Amount,Awarding Agency Code,Awarding Sub Agency Code,Contract Award Type,Award Type,Funding Agency Code,Funding Sub Agency Code,COVID-19 Outlays,COVID-19 Obligations
Recipient DUNS Number,,,,,,,,,,,,,,,,
792283272,YY01,CONT_AWD_YY01_9700_W81K0409D0070_9700,"ADVANCIA AERONAUTICS, LLC",2009-08-13,2009-08-13,2014-08-31,KS,1438924.40,97,2100,DO,NaN,97.0,2100,NaN,NaN
792283272,YY01,CONT_AWD_YY01_9700_W81K0409D0066_9700,"ADVANCIA AERONAUTICS, LLC",2010-06-14,2010-06-11,2012-09-30,OK,870560.35,97,2100,DO,NaN,97.0,2100,NaN,NaN
080409325,W91ZRU21C0002,CONT_AWD_W91ZRU21C0002_9700_-NONE-_-NONE-,"AHTNA GLOBAL, LLC",2020-12-07,2020-12-12,2021-12-11,AK,134756.00,97,2100,DEFINITIVE CONTRACT,NaN,97.0,2100,NaN,NaN
080409325,W91ZRU19P5014,CONT_AWD_W91ZRU19P5014_9700_-NONE-_-NONE-,"AHTNA GLOBAL, LLC",2019-09-26,2019-09-26,2021-11-30,AK,308684.00,97,2100,PURCHASE ORDER,NaN,97.0,5700,NaN,NaN
079766453,W91ZRU19P0030,CONT_AWD_W91ZRU19P0030_9700_-NONE-_-NONE-,"ARCTICOM, LLC",2019-09-26,2019-09-26,2019-12-25,AK,94270.50,97,2100,PURCHASE ORDER,NaN,97.0,2100,NaN,NaN


In [14]:
# set up awards list
awards_list=[]
awards_list.clear()
skipped_list=[]
skipped_list.clear()

In [15]:
# pull award id's from awards_df to use in new api loop
award_ids = awards_df['generated_internal_id'].tolist()

In [16]:
url = "https://api.usaspending.gov/api/v2/awards/"

for i in award_ids:
    award_id = i
    query_url = url + award_id
    award_data = requests.get(query_url)
    print(award_data)
    if award_data.status_code == 200:
        time.sleep(2)
        response_data = award_data.json()
        generated_unique_award_id = response_data["generated_unique_award_id"]
        recip_business_cats = response_data['recipient']['business_categories']
        set_aside = response_data['latest_transaction_contract_data']['type_set_aside']
        set_aside_description = response_data['latest_transaction_contract_data']['type_set_aside_description']
        naics = response_data['latest_transaction_contract_data']['naics']
        naics_description = response_data['latest_transaction_contract_data']['naics_description']
        recipient_parent = response_data['recipient']['parent_recipient_name']
        recipient_parent_duns = response_data['recipient']['parent_recipient_unique_id']
        pop_city = response_data['place_of_performance']['city_name']
        last_mod_date = response_data['period_of_performance']['last_modified_date']
        potential_end_date = response_data['period_of_performance']['potential_end_date']
        total_obligation = response_data['total_obligation']
        awards_list.append({"generated_internal_id":generated_unique_award_id,
                            "Business Categories":recip_business_cats,
                           "Set Aside Type":set_aside,
                           "Set Aside Description":set_aside_description,
                           "NAICS":naics,
                           "NAICS Description":naics_description,
                           "Recipient Parent Name":recipient_parent,
                           "Recipient Parent Duns":recipient_parent_duns,
                           "Place of Performance City":pop_city,
                           "Last Modified Date":last_mod_date,
                            "Potential End Date":potential_end_date,
                           "Total Obligated Amount":total_obligation})
    else: 
        time.sleep(120)
        award_id = i
        query_url = url + award_id
        award_data = requests.get(query_url)
        print(award_data)
        if award_data.status_code == 200:
            response_data = award_data.json()
            generated_unique_award_id = response_data["generated_unique_award_id"]
            recip_business_cats = response_data['recipient']['business_categories']
            set_aside = response_data['latest_transaction_contract_data']['type_set_aside']
            set_aside_description = response_data['latest_transaction_contract_data']['type_set_aside_description']
            naics = response_data['latest_transaction_contract_data']['naics']
            naics_description = response_data['latest_transaction_contract_data']['naics_description']
            recipient_parent = response_data['recipient']['parent_recipient_name']
            recipient_parent_duns = response_data['recipient']['parent_recipient_unique_id']
            pop_city = response_data['place_of_performance']['city_name']
            last_mod_date = response_data['period_of_performance']['last_modified_date']
            potential_end_date = response_data['period_of_performance']['potential_end_date']
            total_obligation = response_data['total_obligation']
            awards_list.append({"generated_internal_id":generated_unique_award_id,
                                "Business Categories":recip_business_cats,
                               "Set Aside Type":set_aside,
                               "Set Aside Description":set_aside_description,
                               "NAICS":naics,
                               "NAICS Description":naics_description,
                               "Recipient Parent Name":recipient_parent,
                               "Recipient Parent Duns":recipient_parent_duns,
                               "Place of Performance City":pop_city,
                               "Last Modified Date":last_mod_date,
                                "Potential End Date":potential_end_date,
                               "Total Obligated Amount":total_obligation})

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

KeyboardInterrupt: 

In [17]:
awards_list

[{'generated_internal_id': 'CONT_AWD_YY01_9700_W81K0409D0070_9700',
  'Business Categories': ['Category Business',
   'Federally Funded Research and Development Corp',
   'Limited Liability Corporation',
   'Minority Owned Business',
   'Small Business',
   'Small Disadvantaged Business',
   'Special Designations',
   'Tribally Owned Firm'],
  'Set Aside Type': 'SBA',
  'Set Aside Description': 'SMALL BUSINESS SET ASIDE - TOTAL',
  'NAICS': '621111',
  'NAICS Description': 'OFFICES OF PHYSICIANS (EXCEPT MENTAL HEALTH SPECIALISTS)',
  'Recipient Parent Name': 'ADVANCIA AERONAUTICS LLC',
  'Recipient Parent Duns': '792283272',
  'Place of Performance City': 'FORT LEAVENWORTH',
  'Last Modified Date': '2013-08-08 00:00:00',
  'Potential End Date': '2014-08-31 00:00:00',
  'Total Obligated Amount': 1438924.4},
 {'generated_internal_id': 'CONT_AWD_YY01_9700_W81K0409D0066_9700',
  'Business Categories': ['Category Business',
   'Federally Funded Research and Development Corp',
   'Limited Li

In [19]:
awards_list_df = pd.DataFrame(awards_list, columns=["generated_internal_id",
                                "Business Categories",
                               "Set Aside Type",
                               "Set Aside Description",
                               "NAICS",
                               "NAICS Description",
                               "Recipient Parent Name",
                               "Recipient Parent Duns",
                               "Place of Performance City",
                               "Last Modified Date",
                                "Potential End Date",
                               "Total Obligated Amount"])

In [20]:
awards_list_df

,generated_internal_id,Business Categories,Set Aside Type,Set Aside Description,NAICS,NAICS Description,Recipient Parent Name,Recipient Parent Duns,Place of Performance City,Last Modified Date,Potential End Date,Total Obligated Amount
0,CONT_AWD_YY01_9700_W81K0409D0070_9700,"[Category Business, Federally Funded Research ...",SBA,SMALL BUSINESS SET ASIDE - TOTAL,621111,OFFICES OF PHYSICIANS (EXCEPT MENTAL HEALTH SP...,ADVANCIA AERONAUTICS LLC,792283272,FORT LEAVENWORTH,2013-08-08 00:00:00,2014-08-31 00:00:00,1438924.40
1,CONT_AWD_YY01_9700_W81K0409D0066_9700,"[Category Business, Federally Funded Research ...",SBA,SMALL BUSINESS SET ASIDE - TOTAL,621111,OFFICES OF PHYSICIANS (EXCEPT MENTAL HEALTH SP...,ADVANCIA AERONAUTICS LLC,792283272,FORT SILL,2012-11-08 00:00:00,2012-09-30 00:00:00,870560.35
2,CONT_AWD_W91ZRU21C0002_9700_-NONE-_-NONE-,"[8(a) Program Participant, Alaskan Native Corp...",8AN,8(A) SOLE SOURCE,541611,ADMINISTRATIVE MANAGEMENT AND GENERAL MANAGEME...,AHTNA GLOBAL LLC,069586055,FORT RICHARDSON,2021-02-18 16:18:51,2025-12-11 00:00:00,134756.00
3,CONT_AWD_W91ZRU19P5014_9700_-NONE-_-NONE-,"[8(a) Program Participant, Alaskan Native Corp...",8AN,8(A) SOLE SOURCE,561320,TEMPORARY HELP SERVICES,AHTNA GLOBAL LLC,069586055,JBER,2021-05-27 16:23:02,2021-11-30 00:00:00,308684.00
4,CONT_AWD_W91ZRU19P0030_9700_-NONE-_-NONE-,"[8(a) Program Participant, Alaskan Native Corp...",8AN,8(A) SOLE SOURCE,517919,ALL OTHER TELECOMMUNICATIONS,ARCTICOM LLC,079766453,FORT RICHARDSON,2021-10-07 09:37:21,2019-12-25 00:00:00,94270.50
...,...,...,...,...,...,...,...,...,...,...,...,...
1244,CONT_AWD_W912BU10F0084_9700_GS10F0287T_4730,"[Category Business, Corporate Entity Not Tax E...",NONE,NO SET ASIDE USED.,541614,"PROCESS, PHYSICAL DISTRIBUTION, AND LOGISTICS ...",AMERICA'S STAFFING PARTNER INC.,800220415,PHILADELPHIA,2020-09-28 09:08:53,2011-09-30 00:00:00,118118.93
1245,CONT_AWD_W9128Z19C0004_9700_-NONE-_-NONE-,"[8(a) Program Participant, Category Business, ...",SBA,SMALL BUSINESS SET ASIDE - TOTAL,541512,COMPUTER SYSTEMS DESIGN SERVICES,ADVANCED IT CONCEPTS INC.,807675603,FORT HUACHUCA,2021-07-13 17:11:24,2022-08-09 12:08:00,1985958.22
1246,CONT_AWD_W9128Z16C0009_9700_-NONE-_-NONE-,"[8(a) Program Participant, Category Business, ...",8AN,8(A) SOLE SOURCE,541519,OTHER COMPUTER RELATED SERVICES,ADVANCED IT CONCEPTS INC.,807675603,FORT BELVOIR,2019-06-05 13:23:13,2018-03-30 00:00:00,2441337.86
1247,CONT_AWD_W9128Z16C0008_9700_-NONE-_-NONE-,"[8(a) Program Participant, Category Business, ...",8AN,8(A) SOLE SOURCE,334112,COMPUTER STORAGE DEVICE MANUFACTURING,ADVANCED IT CONCEPTS INC.,807675603,FORT BELVOIR,2018-04-03 19:44:28,2017-05-31 12:05:00,6623.05


In [25]:
new_merged_df = pd.merge(awards_df,awards_list_df,on='generated_internal_id')
new_merged_df

,Award ID,generated_internal_id,Recipient Name,Base Obligation Date,Start Date,End Date,Place of Performance State Code,Award Amount,Awarding Agency Code,Awarding Sub Agency Code,...,Set Aside Type,Set Aside Description,NAICS,NAICS Description,Recipient Parent Name,Recipient Parent Duns,Place of Performance City,Last Modified Date,Potential End Date,Total Obligated Amount
0,YY01,CONT_AWD_YY01_9700_W81K0409D0070_9700,"ADVANCIA AERONAUTICS, LLC",2009-08-13,2009-08-13,2014-08-31,KS,1438924.40,97,2100,...,SBA,SMALL BUSINESS SET ASIDE - TOTAL,621111,OFFICES OF PHYSICIANS (EXCEPT MENTAL HEALTH SP...,ADVANCIA AERONAUTICS LLC,792283272,FORT LEAVENWORTH,2013-08-08 00:00:00,2014-08-31 00:00:00,1438924.40
1,YY01,CONT_AWD_YY01_9700_W81K0409D0066_9700,"ADVANCIA AERONAUTICS, LLC",2010-06-14,2010-06-11,2012-09-30,OK,870560.35,97,2100,...,SBA,SMALL BUSINESS SET ASIDE - TOTAL,621111,OFFICES OF PHYSICIANS (EXCEPT MENTAL HEALTH SP...,ADVANCIA AERONAUTICS LLC,792283272,FORT SILL,2012-11-08 00:00:00,2012-09-30 00:00:00,870560.35
2,W91ZRU21C0002,CONT_AWD_W91ZRU21C0002_9700_-NONE-_-NONE-,"AHTNA GLOBAL, LLC",2020-12-07,2020-12-12,2021-12-11,AK,134756.00,97,2100,...,8AN,8(A) SOLE SOURCE,541611,ADMINISTRATIVE MANAGEMENT AND GENERAL MANAGEME...,AHTNA GLOBAL LLC,069586055,FORT RICHARDSON,2021-02-18 16:18:51,2025-12-11 00:00:00,134756.00
3,W91ZRU19P5014,CONT_AWD_W91ZRU19P5014_9700_-NONE-_-NONE-,"AHTNA GLOBAL, LLC",2019-09-26,2019-09-26,2021-11-30,AK,308684.00,97,2100,...,8AN,8(A) SOLE SOURCE,561320,TEMPORARY HELP SERVICES,AHTNA GLOBAL LLC,069586055,JBER,2021-05-27 16:23:02,2021-11-30 00:00:00,308684.00
4,W91ZRU19P0030,CONT_AWD_W91ZRU19P0030_9700_-NONE-_-NONE-,"ARCTICOM, LLC",2019-09-26,2019-09-26,2019-12-25,AK,94270.50,97,2100,...,8AN,8(A) SOLE SOURCE,517919,ALL OTHER TELECOMMUNICATIONS,ARCTICOM LLC,079766453,FORT RICHARDSON,2021-10-07 09:37:21,2019-12-25 00:00:00,94270.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1244,W912BU10F0084,CONT_AWD_W912BU10F0084_9700_GS10F0287T_4730,"AMERICA'S STAFFING PARTNER, INC.",2010-09-17,2010-09-17,2011-09-30,PA,118118.93,97,2100,...,NONE,NO SET ASIDE USED.,541614,"PROCESS, PHYSICAL DISTRIBUTION, AND LOGISTICS ...",AMERICA'S STAFFING PARTNER INC.,800220415,PHILADELPHIA,2020-09-28 09:08:53,2011-09-30 00:00:00,118118.93
1245,W9128Z19C0004,CONT_AWD_W9128Z19C0004_9700_-NONE-_-NONE-,"ADVANCED IT CONCEPTS, INC.",2019-07-17,2019-08-10,2022-08-09,AZ,1985958.22,97,2100,...,SBA,SMALL BUSINESS SET ASIDE - TOTAL,541512,COMPUTER SYSTEMS DESIGN SERVICES,ADVANCED IT CONCEPTS INC.,807675603,FORT HUACHUCA,2021-07-13 17:11:24,2022-08-09 12:08:00,1985958.22
1246,W9128Z16C0009,CONT_AWD_W9128Z16C0009_9700_-NONE-_-NONE-,"ADVANCED IT CONCEPTS, INC.",2016-09-29,2016-09-29,2018-03-30,VA,2441337.86,97,2100,...,8AN,8(A) SOLE SOURCE,541519,OTHER COMPUTER RELATED SERVICES,ADVANCED IT CONCEPTS INC.,807675603,FORT BELVOIR,2019-06-05 13:23:13,2018-03-30 00:00:00,2441337.86
1247,W9128Z16C0008,CONT_AWD_W9128Z16C0008_9700_-NONE-_-NONE-,"ADVANCED IT CONCEPTS, INC.",2016-09-28,2016-09-29,2017-05-31,VA,6623.05,97,2100,...,8AN,8(A) SOLE SOURCE,334112,COMPUTER STORAGE DEVICE MANUFACTURING,ADVANCED IT CONCEPTS INC.,807675603,FORT BELVOIR,2018-04-03 19:44:28,2017-05-31 12:05:00,6623.05


In [27]:
new_merged_df.to_csv('new_merged_df.csv')